
<font color=blue|red|green|pink|yellow> **DATA 900** </font>
 
## **Claims Data Assignment**

<font color=blue|red|green|pink|yellow> **Nemshan ** </font>

<img src=http://www.actionforbetterhealthcare.com/wp-content/uploads/2014/11/400-04794931b-590x260.jpg></img>


*Medical claims file for 2016 contains ~17 millions rows and ~60 columns of data, containing ~6.5 million individual medical claims. These claims are all commercial claims that were filed by healthcare providers in 2016 in the state of NH. These claims were ~88% for residents of NH and the remaining for out of state visitors who sought care in NH. Each claim consists of one or more line items, each indicating a procedure done during the doctor’s visit. Two columns indicating Billed amount and the Paid amount for the care provided, are of primary interest.*
<font color=blue|red|green|pink|yellow> The main objective is to predict “Paid amount per procedure” by mapping a plethora of features available in the dataset </font>

**My approach solving this problem was through 3 steps:**

### 1. first I used the chunking technique because I had a big file " 3.73 GB." and I chunked that file into small peces to put them into one dataframe.
   + I wrote to CSV file so that I can use it later.
   + created CSV  file with 1,000,000 sampled claim ids
   + joined the data into a subsetted data file
### 2. Data cleaning and exploration
    
    + I removed 51 columns for these reasons : 
         + 63 features are too many to predict with
         + if the column has a lot of missing data."more than 2 millon"
         + if I think it would not add predictive value to the model
         + I used JMP to look at the distributions of the features and found some of the features had a lot of just one value which seems not useful to be used in predicting. 
    + I generalized the Icd_code into small numbers of categories based on the starting letters of code because it has a lot of categories
    + I thought the client_los would be a good predictor, but it has a lot of missing data. SO I assumed that the missing data were zeros.
    + I made indicator variable of gender by changing M  to 1 and F to 0.
    + before I removed diagnosis columns, I counted how many diagnoses each claim has and made that into a new column.
    + I changed all categorical variables to dummy variables.
    + also I removed a few rows that have a missing value. 
### 3. modling :-
    + I started with liner models lasso and ridge. In this data, I thought the liner models would not do well and I didn't expect to see a linear trend. After running lasso and rdge a got a low R^2. 
    + I used nonlinear models ( MARS, Random forest and Adaboost)

        + MARS : 
        I ran Earth model with different hyper-parameters. I chose the best model with the best R^2 score.

        + Random forest :
        I tried a lot of different hyper parameter to tune the model. I saw that 300 trees were good enough to run the model without taking a lot of time. But at the end, 44% as the best R^2 score I got which seems not a good model.
        + Adaboost :
        After trying a different number of estimators for  AdaBoost, the model I got was with 200 estimators. And it seems this modeling algorithm is not the best choice to use for this data. 



## more explation of each step throuh out the code :

In [18]:


import pandas as pd
import math
import warnings
import random
import time
warnings.filterwarnings("ignore")

folder_path = '/use the directory file path/'

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# open the whole file
full_file = open(folder_path + 'PUBLICUSE_CLAIM_MC_2016.txt', 'r')

# keep track of how long the process takes
start_time = time.time()

# change this to try a different number of rows per chunk
chunk_rows = 100000

num_chunks = math.ceil(16982295/chunk_rows)
print('\nChunking data with chunk size: ', chunk_rows, sep = '')
print('Number of chunks to be used:', num_chunks, '\n')

# chunking data using read_csv chunksize parameter
small_df = pd.read_csv(full_file, sep = '|', header = 0, chunksize = chunk_rows)

# chunks the data into one pandas df
full_df = pd.concat(small_df, ignore_index=True)

print('Writing data to file...\n')
output_path = '/use the directory file path/'
full_df.to_csv(output_path + 'full_data.csv', index = False)
print('Length of subsetted df:', len(full_df), '\n')

elapsed_time = time.time() - start_time
m, s = divmod(elapsed_time, 60)
time = "%02d:%02d" % (m, s)
print('Time elapsed:', time, '\n')






Chunking data with chunk size: 100000
Number of chunks to be used: 170 

Writing data to file...

Length of subsetted df: 16982295 

Time elapsed: 17:22 



In [5]:

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# create csv with 1,000,000 sampled claim ids
claims_set = set() # create empty set to append unique claims to
print('Finding unique claim IDs...\n')
for id_num in full_df.iloc[:, 4]:
    if id_num not in claims_set:
        claims_set.add(id_num)

claims_list = list(claims_set)

million_claims_list = random.sample(claims_list, 1000000)
million_claims_df = pd.DataFrame(million_claims_list, columns=["CLAIM_ID_KEY"])

print('Number of unique claim IDs:', len(claims_list))
print('Number of claim IDs sampled:', len(million_claims_list), '\n')

output_path = '/use the directory file path/'
million_claims_df.to_csv(output_path + 'one_million_claims.csv', index=False)


Finding unique claim IDs...

Number of unique claim IDs: 7030309
Number of claim IDs sampled: 1000000 



In [6]:

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# join data into a subsetted data file
path = '/use the directory file path/'

subset_df = pd.DataFrame()

million_claims = pd.read_csv(path + 'one_million_claims.csv', sep = ",")
for i in range(1, 17):
    j = (i + 1) * 1000000
    lista = list(range(1, j))

    claims_file = pd.read_csv(path + 'full_data.csv', sep = ',', nrows = 1000000, skiprows = lista)
    mergedata = pd.merge(million_claims, claims_file, how = 'inner', on = ['CLAIM_ID_KEY'])
    subset_df = pd.concat([subset_df, mergedata])

    print("Finished chunk", i, 'of 17.')
    print('Number of Claims:', len(claims_file), '- Number of IDs:', len(million_claims), '\n')

print('\nWriting data to csv...\n', sep = '')
subset_df.to_csv(path + 'subsetted_data.csv', index = False)
print('Subsetted data file length: ', len(subset_df), sep = '')


Finished chunk 1 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 2 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 3 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 4 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 5 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 6 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 7 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 8 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 9 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 10 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 11 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 12 of 17.
Number of Claims: 1000000 - Number of IDs: 1000000 

Finished chunk 13 of 17.
Number of Claims: 1000000 - Number of IDs: 10000

In [8]:

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# testing to make sure data looks right
print('\n# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #' , sep = '')
print('Testing subsetted dataframe...')
data_path = '/use the directory file path/'

testing_df = pd.read_csv(data_path + 'subsetted_data.csv', sep = ',')
print('Length of subsetted data:', len(testing_df))

claims_set_test = set()
for id_num in testing_df.loc[:, 'CLAIM_ID_KEY']:
    if id_num not in claims_set_test:
        claims_set_test.add(id_num)
print('Number of unique claim IDs:', len(claims_set_test))
print('Number of columns:', len(list(testing_df)), '\n')

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# data cleaning and prepation and exploration
sub_data = testing_df.loc[:, ['CLAIM_ID_KEY', 'ICD_DIAG_01_PRIMARY', 'ICD_DIAG_02', 'ICD_DIAG_03', 'ICD_DIAG_04', 'ICD_DIAG_05', 'ICD_DIAG_06', 'ICD_DIAG_07', 'ICD_DIAG_07', 'ICD_DIAG_08', 'ICD_DIAG_09', 'ICD_DIAG_10', 'ICD_DIAG_11', 'ICD_DIAG_12', 'ICD_DIAG_13']]
sub_cols = list(sub_data)
sub_data['Num_diag'] = sub_data[sub_cols].count(axis = 1) - 1
sub_data1 = sub_data['Num_diag']
testing_df = pd.concat([testing_df, sub_data1], axis = 1)

# print(testing_df.isnull().sum() > 1000000)
missing_vals = testing_df.isnull().sum()
cols = []
rm_cols_list = []

# make list of all columns
for index, val in enumerate(missing_vals):
    cols.append(testing_df.columns[index])

print('\nColumns with <1,000,000 missing values: ', sep = '')
for index, val in enumerate(missing_vals):
    if val < 1000000:
        print(testing_df.columns[index], ': ', val, sep = '')

print('\nColumns with >1,000,000 missing values: ', sep = '')
for index, val in enumerate(missing_vals):
    if val > 1000000:
        print(testing_df.columns[index], ': ', val, sep = '')
        rm_cols_list.append(testing_df.columns[index])

print('\nColumns with >2,000,000 missing values: ', sep = '')
for index, val in enumerate(missing_vals):
    if val > 2000000:
        print(testing_df.columns[index], ': ', val, sep = '')

rm_cols_list.remove('CLIENT_LOS')

testing_df = testing_df.drop(rm_cols_list,axis =1)

for item in cols:
    if item in rm_cols_list:
        cols.remove(item)

print('\nNew number of columns: ', len(list(testing_df)), sep = '')
print('Columns removed:', '\n', rm_cols_list, '\n')
print('Columns remaining:', '\n', cols, '\n')
print('Length of subsetted data (again):', len(testing_df))

# save to csv for more cleaning and faster loading
print('Saving data to new csv...\n')
testing_df.to_csv(data_path + 'data_cleaning_step1.csv', index = False)
print('Save complete.\n')

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #





# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
Testing subsetted dataframe...
Length of subsetted data: 2131941
Number of unique claim IDs: 935073
Number of columns: 63 


Columns with <1,000,000 missing values: 
CLAIM_ID_KEY: 0
COVERAGE_CLASS: 0
FROM_YEAR: 0
ADM_YR: 0
DIS_YR: 0
CS_CLAIM_ID_KEY: 0
SERVICES_KEY: 0
SV_LINE: 0
FORM_TYPE: 0
SV_STAT: 0
POS: 0
AGE: 0
SEX: 0
MEMBER_COUNTY: 0
MEMBER_STATE: 0
PRODUCT_TYPE: 45867
LOB: 0
INSURANCE_TYPE: 0
PROC_CODE: 154614
CPT_MOD1: 0
CPT_MOD2: 0
ICD_10_OR_HIGHER: 10003
ICD_DIAG_01_PRIMARY: 10003
ICD_DIAG_02: 940663
SERV_PROV_CW_KEY: 51
BILL_PROV_CW_KEY: 51
QTY: 0
AMT_BILLED: 0
AMT_PAID: 0
AMT_DEDUCT: 0
AMT_COINS: 0
AMT_COPAY: 0
AMT_PREPAID: 0
INPATIENT_FLAG: 0
MR_LINE_CASE_KEY: 0
UTILS: 710770
CLAIM_STATUS_ORIG: 0
ADMIT_HOUR: 380969
DISCHARGE_HOUR: 185706
CLAIM_ADJUSTMENT_LOGIC: 0
IMPUTED_SERVICE_KEY: 0
Num_diag: 0

Columns with >1,000,000 missing values: 
DIS_STAT: 1446012
REV_CODE: 1446668
UB_BILL_TYPE: 1446475
ADM_SRC: 1

In [23]:
import pandas as pd
import time
import warnings
import numpy as np
warnings.filterwarnings("ignore")


data_path = '/use the directory file path/'

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# further cleaning of data
# start_time = time.time()
data = pd.read_csv(data_path + 'data_cleaning_step1.csv', sep = ',')
print('\nStarting number of columns: ', len(list(data)), sep = '')

# removing columns decided ahead of time
cols_to_remove = ['FROM_YEAR', 'ADM_YR', 'DIS_YR', 'SERVICES_KEY', 'SV_LINE', 'MEMBER_STATE', 'INSURANCE_TYPE', 'CPT_MOD1', 'CPT_MOD2', 'ICD_10_OR_HIGHER', 'QTY', 'AMT_COPAY', 'AMT_PREPAID', 'INPATIENT_FLAG', 'UTILS', 'CLAIM_STATUS_ORIG', 'ADMIT_HOUR', 'DISCHARGE_HOUR', 'CLAIM_ADJUSTMENT_LOGIC', 'IMPUTED_SERVICE_KEY']

data = data.drop(cols_to_remove,axis =1)
print('Number of columns remaining:', len(list(data)), '\n')
print('Data types:', '\n', data.dtypes, '\n')

# make age numeric , make gender 1 or 0, proc code to categories of letters, pos to dummies 

# gender
print('Changing SEX to Gender_Code and dropping original column...\n')
data['Gender_Code'] = data['SEX'].apply(lambda x: 1 if x == 'M' else 0)
data = data.drop(['SEX'], axis=1)

# proc code into letter categories
print('Changing PROC_CODE to Proc_Code_letters and dropping original column...\n')
data['Proc_Code_letters'] = data['PROC_CODE'].apply(lambda x: str(x)[0])
data = data.drop(['PROC_CODE'], axis=1)

# getting rid of categorical age - making 90+ into 90
print('Changing AGE to Age and dropping original column...\n')
data['Age'] = data['AGE'].apply(lambda x: 90 if x == '90+' else x)
data['Age'] = data['Age'].astype(str).astype(int)
data = data.drop( ['AGE'], axis=1)

# add zeros to missing CLIENT_LOS
print('Adding zeros for missing values in CLIENT_LOS...\n')
data['CLIENT_LOS'] = data['CLIENT_LOS'].apply(lambda x: 0 if pd.isnull(x) else x)

# summary after cleaning
print('Number of columns remaining:', len(list(data)), '\n')
print('Data types:', '\n', data.dtypes, '\n', sep = '')

# output to csv again ... smaller this time
print('Writing new csv...')
data.to_csv(data_path + 'clean_step_3.csv', index = False)

# elapsed_time = time.time() - start_time
# m, s = divmod(elapsed_time, 60)
# time = "%02d:%02d" % (m, s)
# print('\nTime elapsed: ', time, '\n', sep = '')

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# cleaning from cleaner file
# start_time = time.time()
#print('Opening data file...\n')
data2 = pd.read_csv(data_path + 'clean_step_3.csv', sep = ',')

# check missing values for each column
missing_vals = data2.isnull().sum()
print('Missing values per column:\n', missing_vals, '\n', sep = '')

# test if these two columns are equal
print('Are BILL_PROV_CW_KEY and SERV_PROV_CW_KEY equal?\n')
print((data2['SERV_PROV_CW_KEY'] == data2['BILL_PROV_CW_KEY']).value_counts())
# neither actually seem too helpful anyways - drop one for now - come back for the other?
data2 = data2.drop(['BILL_PROV_CW_KEY'], axis=1)

# test if these two columns are equal
print('Are CLAIM_ID_KEY and CS_CLAIM_ID_KEY equal?\n')
print((data2['CLAIM_ID_KEY'] == data2['CS_CLAIM_ID_KEY']).value_counts())
data2 = data2.drop(['CS_CLAIM_ID_KEY'], axis=1)


# ICD_DIAG_02 has almost half missing values
print('Dropping ICD_DIAG_01_PRIMARY column...\n')
data2 = data2.drop(['ICD_DIAG_02'], axis=1)

# change ICD_DIAG_01_PRIMARY to just first letters
print('Changing ICD_DIAG_01_PRIMARY column...\n')
print('Changing ICD_DIAG_01_PRIMARY to ICD_DIAG_01_PRIMARY_categories and dropping original column...\n')
data2['ICD_DIAG_01_PRIMARY_categories'] = data2['ICD_DIAG_01_PRIMARY'].apply(lambda x: str(x)[0])
data2 = data2.drop(['ICD_DIAG_01_PRIMARY'], axis=1)


# test if these two columns are equal
print('Are ICD_DIAG_01_PRIMARY_categories and Proc_Code_letters equal?\n')
print((data2['ICD_DIAG_01_PRIMARY_categories'] == data2['Proc_Code_letters']).value_counts())


# drop all rows with null values
print('Dropping all rows with null values...\n')
data2 = data2.dropna(axis = 0, how = 'any')

# drop all rows with negaitive billing amounts and paid amounts
print('Dropping rows with negative bill and paid amounts...\n')
data2 = data2[data2['AMT_BILLED'] >= 0]
data2 = data2[data2['AMT_PAID'] >= 0]


# check missing values for each column before writing to csv
missing_vals = data2.isnull().sum()
print('Missing values per column:\n', missing_vals, '\n', sep = '')
print('Number of columns remaining:', len(list(data2)), '\n')

# output to csv again
print('Writing new csv...')
data2.to_csv(data_path + 'clean_step_4.csv', index = False)

# elapsed_time = time.time() - start_time
# m, s = divmod(elapsed_time, 60)
# time = "%02d:%02d" % (m, s)
# print('\nTime elapsed: ', time, '\n', sep = '')


# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# cleaning more - dropping columns
# start_time = time.time()
print('Opening data file...\n')
data3 = pd.read_csv(data_path + 'clean_step_4.csv', sep = ',')

# drop CLAIM_ID_KEY, MEMBER_COUNTY, SERV_PROV_CW_KEY, MR_LINE_CASE_KEY, Proc_Code_letters, COVERAGE_CLASS, LOB
print('Dropping columns...\n')
data3 = data3.drop( ['CLAIM_ID_KEY'],axis=1)
data3 = data3.drop(['MEMBER_COUNTY'],axis=1)
data3 = data3.drop(['SERV_PROV_CW_KEY'],axis=1)
data3 = data3.drop( ['MR_LINE_CASE_KEY'],axis=1)
data3 = data3.drop( ['Proc_Code_letters'],axis=1)
data3 = data3.drop( ['COVERAGE_CLASS'],axis=1)
data3 = data3.drop( ['LOB'],axis=1)
data3 = data3.drop( ['POS'],axis=1)

# drop negative coins and deduct amounts
print('Dropping rows with negative bill and paid amounts...\n')
data3 = data3[data3['AMT_COINS'] >= 0]
data3 = data3[data3['AMT_DEDUCT'] >= 0]

missing_vals = data3.isnull().sum()
print('Missing values per column:\n', missing_vals, '\n', sep = '')
print('Number of columns remaining:', len(list(data3)))
print('Length of data:', len(data3), '\n')


# combine some of the icd10 codes?
print('Merging some ICD categories...\n')
data3['ICD_DIAG_01_PRIMARY_categories'] = data3['ICD_DIAG_01_PRIMARY_categories'].apply(lambda x: 'V' if x == 'W' else x)
data3['ICD_DIAG_01_PRIMARY_categories'] = data3['ICD_DIAG_01_PRIMARY_categories'].apply(lambda x: 'V' if x == 'X' else x)
data3['ICD_DIAG_01_PRIMARY_categories'] = data3['ICD_DIAG_01_PRIMARY_categories'].apply(lambda x: 'V' if x == 'Y' else x)
data3['ICD_DIAG_01_PRIMARY_categories'] = data3['ICD_DIAG_01_PRIMARY_categories'].apply(lambda x: 'A' if x == 'B' else x)

data3['ICD_DIAG_01_PRIMARY_categories'] = data3['ICD_DIAG_01_PRIMARY_categories'].apply(lambda x: 'num' if x.isdigit() else x)

data3['ICD_DIAG_01_PRIMARY_categories'] = data3['ICD_DIAG_01_PRIMARY_categories'].apply(lambda x: 'N' if x == 'n' else x)


# output to csv again
print('Writing new csv...')
data3.to_csv(data_path + 'final_clean_data2.csv', index = False)
print('Save complete.')

# elapsed_time = time.time() - start_time
# m, s = divmod(elapsed_time, 60)
# time = "%02d:%02d" % (m, s)
# print('\nTime elapsed: ', time, '\n', sep = '')



Starting number of columns: 43
Number of columns remaining: 23 

Data types: 
 CLAIM_ID_KEY             int64
COVERAGE_CLASS          object
CS_CLAIM_ID_KEY        float64
FORM_TYPE               object
SV_STAT                 object
POS                     object
AGE                     object
SEX                     object
MEMBER_COUNTY          float64
PRODUCT_TYPE            object
LOB                     object
PROC_CODE               object
CLIENT_LOS             float64
ICD_DIAG_01_PRIMARY     object
ICD_DIAG_02             object
SERV_PROV_CW_KEY       float64
BILL_PROV_CW_KEY       float64
AMT_BILLED             float64
AMT_PAID               float64
AMT_DEDUCT             float64
AMT_COINS              float64
MR_LINE_CASE_KEY       float64
Num_diag                 int64
dtype: object 

Changing SEX to Gender_Code and dropping original column...

Changing PROC_CODE to Proc_Code_letters and dropping original column...

Changing AGE to Age and dropping original column...

Addi

In [2]:
#modeling step:
import pandas as pd
import time

start_time = time.time()

print('Opening data file...')
data_path = '/use the directory file path/'
df = pd.read_csv(data_path + 'final_clean_data2.csv', index_col = False)
print('Data open.')
print('Starting number of columns:', len(list(df)))

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# make dummies and normalize data
from scipy import stats
from scipy.stats import zscore

df_dummies = pd.get_dummies(df, columns=['FORM_TYPE', 'SV_STAT', 'PRODUCT_TYPE', 'ICD_DIAG_01_PRIMARY_categories'])

unstandardized_df = df_dummies

# Standardize
cols = ['CLIENT_LOS', 'AMT_BILLED','AMT_PAID', 'AMT_DEDUCT',
         'AMT_COINS', 'Num_diag', 'Age']

normalized_df = df_dummies[cols].apply(zscore)
for col in cols:
    df_dummies = df_dummies.drop( [col],axis=1)

normalized_df = pd.concat([df_dummies, normalized_df], axis = 1)

print('New number of columns:', len(list(normalized_df)), '\n')

# print('Columns used:')
# for item in list(normalized_df):
#     print(item)
# print('\n')

df = normalized_df

# print('df columns:\n', list(df), sep = '')
# print(len(list(df)))

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# train-test split
from sklearn.model_selection import train_test_split

print('Splitting data...\n')
X = normalized_df.loc[:, normalized_df.columns != 'AMT_PAID']
y = normalized_df.loc[:, 'AMT_PAID']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 111)
# print('X_train length:', len(X_train), 'X_test length:', len(X_test))
# print('y_train length:', len(y_train), 'y_test length:', len(y_test))
print('# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #\n')

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# LASSO model
print('Running Lasso model...')
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

alpha_value_lasso = 0.1
lasso = Lasso(alpha = alpha_value_lasso)

y_pred_lasso = lasso.fit(X_train, y_train).predict(X_test)
r2_score_lasso = r2_score(y_test, y_pred_lasso)
print("Lasso r^2 on test data : %f" % r2_score_lasso, '\n')
print('# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #\n')

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# Ridge model
print('Running Ridge model...')
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

alpha_value_ridge = 0.5
ridge = Ridge(alpha = alpha_value_ridge)

y_pred_ridge = ridge.fit(X_train, y_train).predict(X_test)
r2_score_ridge = r2_score(y_test, y_pred_ridge)
print("Ridge r^2 on test data : %f" % r2_score_ridge, '\n')
print('# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #\n')


# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
elapsed_time = time.time() - start_time
m, s = divmod(elapsed_time, 60)
time = "%02d:%02d" % (m, s)
print('\nTime elapsed: ', time, '\n', sep = '')



Opening data file...
Data open.
Starting number of columns: 12
New number of columns: 42 

Splitting data...

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #

Running Lasso model...
Lasso r^2 on test data : 0.122718 

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #

Running Ridge model...
Ridge r^2 on test data : 0.135109 

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #


Time elapsed: 00:14



In [16]:

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# Earth model
from pyearth import Earth
from matplotlib import pyplot
from sklearn.metrics import r2_score

thresh_val = [0.01]
scores = []
for item in thresh_val:
    print('Running Earth model with thresh ', item, '...', sep = '')
    earth_model = Earth(max_degree = 3, use_fast = True, smooth = True, thresh = item, verbose = 2)
    earth_model.fit(X_train, y_train)

    print('Earth Score: %f' % r2_score(y_test, earth_model.predict(X_test)), '\n')
    scores.append(r2_score)
    print('# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #\n')

print('Threshold values:', thresh_val)
print('Scores:', scores)
print()


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     1.018349  1      1.018  0.000  0.000  
1     0       1    52706  0.742267  3      0.742  0.271  0.271  
2     1       9    -1    0.582815  4      0.583  0.428  0.428  
3     1       6    -1    0.545224  5      0.545  0.465  0.465  
4     1       18   -1    0.536123  6      0.536  0.474  0.474  
5     1       7    -1    0.530261  7      0.530  0.479  0.479  
6     1       37   -1    0.525039  8      0.525  0.484  0.484  
7     0       3    1576541  0.520280  10     0.520  0.489  0.489  
------------------------------------------------------------------
Stopping Condition 2: Improvement below threshold
Beginning pruning pass
--------------------------------------------
iter  bf  terms  mse   gcv    rsq    grsq   
---------------------------------------

In [30]:
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #
# Random forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

# re-split data for unstandardized data
from sklearn.model_selection import train_test_split

# print('Splitting data...')
X = unstandardized_df.loc[:, unstandardized_df.columns != 'AMT_PAID']
y = unstandardized_df.loc[:, 'AMT_PAID']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 111)
print('X_train length:', len(X_train), 'X_test length:', len(X_test))
print('y_train length:', len(y_train), 'y_test length:', len(y_test))

#modling using the best values to choose whcih one to use
# depths = [30, 35, 40, 45, 50, 55]
# rf_scores = []
# for item in depths:
#     print('Running random forest model with max_depth = ', item, '...', sep = '')
#     regressor = RandomForestRegressor(n_estimators = 20, max_depth = item, max_features = 'sqrt', n_jobs = 4, oob_score = True, random_state = 1234) #max_depth = 35
#     regressor.fit(X_train, y_train)

#     predicted = regressor.predict(X_test)
#     r2 = r2_score(y_test, predicted)
#     print('Random Forest score:', r2, '\n')
#     rf_scores.append(r2)
#     print('# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #\n')
# for item in depths:
#     print(item, ':', rf_scores[depths.index(item)])

### BEST MODEL: max feaures = sqrt, max_depth = 30-50
print('Running random forest model with optimal settings...', sep = '')
regressor = RandomForestRegressor(n_estimators = 300, max_depth = 30, max_features = 'sqrt', n_jobs = -1, oob_score = True) #max_depth = 35
regressor.fit(X_train, y_train)
predicted = regressor.predict(X_test)
r2 = r2_score(y_test, predicted)
print('Random Forest score:', r2, '\n')


X_train length: 1610698 X_test length: 402675
y_train length: 1610698 y_test length: 402675
Running random forest model with optimal settings...
Random Forest score: 0.436817204208 

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- #



In [17]:

# Ada boosted model
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score

print('Running ADA boosted model...')
ada_boost = AdaBoostRegressor(n_estimators = 200)
ada_boost.fit(X_train, y_train)
prediction = ada_boost.predict(X_test)
r2 = r2_score(y_test, prediction)
# score = ada_boost.score(X_test, )
print('ADA boosted regression model score:', r2, '\n')





Running ADA boosted model...
ADA boosted regression model score: 0.227440371675 


# Takeaways from working on this project :
* I learned how to chunk big files.
* I become more familiar with making the data into granule level.
* I could have missed features that important in predicting  Paid amount per procedure because of my limited knowledge of healthcare system. 
* I learned more how to apply some of new the algorithms.